In [3]:
import numpy as np
import scipy
import scipy.integrate
import matplotlib.pyplot as plt

In [ ]:
params = dict()
params['reflected_rotor_inertia'] = 0.01 # kg * m^2
params['m_l'] = 3.0 # kg
params['m_b'] = 32.0 # kg
params['k_i'] = 1e4 # N/m

params['k_impedance'] = 10.0 # N/m spring constant
# params['k_impedance'] = 0.0 # N/m spring constant


params_flow_source = dict()
params_flow_source['amplitude'] = 0.1 # m
params_flow_source['frequency'] = 1.0 # Hz

m_1 = 1.55 # kg mass of Femur
m_2 = 0.157 # kg mass of Tibia
m_3 = 0.166 # mass of Foot
m_l = m_1 + m_2 + m_3
print "mass of leg from paper", m_l

J_motor_cheetah = 3.0279 * 1e-4
gr_cheetah = 5.8
T_cheetah = 1/gr_cheetah
J_motor_reflected = gr_cheetah**2 * J_motor_cheetah
print "J_motor_reflected", J_motor_reflected

In [ ]:
# simulate the system implementing impedance control

class Cheetah_1_dof(object):
    
    def __init__(self, params, params_flow_source):
        self._params = params
        self._params_flow_source = params_flow_source
        
    
    def _setup(self):
        self._I = self._params['m_l'] + self._params['reflected_rotor_inertia']
        
    def dydt(self, t, y):
        """
        y = [x, v] is the state vector
        """
#         self._update(t,y)
        x = y[0]
        v = y[1]
        dxdt = v
        dvdt = self.dvdt(t, y)
        dydt = np.array([dxdt, dvdt])
        
        return dydt
    
    def dvdt(self, t, y):
        x = y[0]
        v = y[1]
        tau = self.motor_torque(t, y)
        x_g = self.ground_position(t, y)
        v_dot = (1/self._I) * (tau - self._params['k_i']*(x-x_g))
        
        return v_dot
        
        
    def motor_torque(self, t, y):
        """
        Returns the desired torque given a position
        """
        tau = self.desired_impedance_force(t,y)
        return tau
        
    def ground_position(self, t, y):
        # ground position as a function of t
        amplitude = self._params_flow_source['amplitude']
        frequency_hz = self._params_flow_source['frequency']
        x_g = - amplitude/2.0*(1 - np.cos(t * 2 * np.pi * frequency_hz))
        return x_g
        
    def simulate(self, t_final, dt = 0.01):
        """
        Simulates the system forward in time
        """
        self._setup()
        y0 = np.array([0,0])
        t_vec = np.arange(0, t_final, dt)
        y = scipy.integrate.odeint(self.dydt, y0, t_vec, tfirst=True)
        return t_vec, y
        
        
    def plot(self, t_vec, y):
        """
        """
        fig, axes = plt.subplots(nrows=2, ncols=1)
        
        # plot position
        ax = axes[0]
        ax.plot(t_vec, y[:, 0])
        ax.set_xlabel('t')
        ax.set_ylabel('x')
        
        # plot velocity
        ax = axes[1]
        ax.plot(t_vec, y[:, 1])
        ax.set_xlabel('t')
        ax.set_ylabel('v')
#         ax.xlabel

        plt.show()
    
    def desired_impedance_force(self, t, y):
        x_g = self.ground_position(t, y)
        force = -self._params['k_impedance'] * x_g
        return force        
    
    def end_effector_force(self, t, y):
        x_g = self.ground_position(t, y)
        x = y[0]
        ee_force = self._params['k_i']*(x-x_g)
        return ee_force

    
def apply_function_elementwise(func, t_vec, y):
    output = None
    N = t_vec.size
    for idx in xrange(0, N):
        res = func(t_vec[idx], y[idx,:])
        if output is None:
            output = np.zeros([N, res.size])
        
        output[idx, :] = res
        
    return output
        
    
    
    
    
# def apply_function_elementwise()
        
        

In [ ]:
t_final = 10
dt = 0.001
model = Cheetah_1_dof(params, params_flow_source)
t_vec, y = model.simulate(t_final, dt)
t_vec = t_vec.reshape([t_vec.size, 1])
ee_force = apply_function_elementwise(model.end_effector_force, t_vec, y)
desired_ee_force = apply_function_elementwise(model.desired_impedance_force, t_vec, y)
x_g_vec = apply_function_elementwise(model.ground_position, t_vec, y)

In [ ]:
print t_vec.shape
print desired_ee_force.shape
model.plot(t_vec, y)

plt.figure()
plt.plot(t_vec, x_g_vec, 'b')
plt.plot(t_vec, y[:, 0], 'r')
plt.ylabel('x_g')
plt.xlabel('t"')

In [ ]:
print x_g_vec.shape
x_g_vec = x_g_vec.squeeze()
print x_g_vec.shape
print y[:, 0].shape
x_s = x_g_vec - y[:,0]
x_s[0]


In [ ]:
print ee_force.shape
x_s = x_g_vec - y[:,0]
plt.plot(t_vec, desired_ee_force, 'b')
plt.show()
plt.plot(t_vec, x_s)
plt.show()

plt.plot(t_vec, params['k_i']*x_s)
plt.show()
# plt.plot(t_vec, ee_force, 'r')

In [5]:
T_1 = 7.66
T_2 = 8.846
print T_1**2 * 0.0005
np.rad2deg(0.05)
0.05*

0.0293378


NameError: name 'k_p' is not defined